In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 8.5 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=88b52afabaf02de902a4c079ef4bdfe1201647124bd4c79485b54c88958215c6
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
requests 2.23.0 requires urllib3!=

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Download 100%.


In [2]:
! signate download --competition-id=406

sample_submission.csv

train.csv

test_data.zip

train_data.zip


Download completed.


In [3]:
! unzip -q train_data.zip 
! unzip -q test_data.zip

In [4]:
! ls

__MACOSX     sample_submission.csv  test_data.zip  train_data
sample_data  test_data		    train.csv	   train_data.zip


In [5]:
!pip install -q omegaconf timm pytorch-metric-learning[with-hooks] pytorch-lightning

     |████████████████████████████████| 79 kB 3.9 MB/s 
     |████████████████████████████████| 549 kB 11.1 MB/s 
     |████████████████████████████████| 111 kB 71.8 MB/s 
     |████████████████████████████████| 800 kB 69.7 MB/s 
     |████████████████████████████████| 117 kB 78.4 MB/s 
     |████████████████████████████████| 182 kB 71.8 MB/s 
     |████████████████████████████████| 125 kB 52.4 MB/s 
     |████████████████████████████████| 512 kB 66.7 MB/s 
     |████████████████████████████████| 85.5 MB 64 kB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
signate 0.9.9 requires urllib3>=1.26.7, but you have urllib3 1.25.11 which is incompatible.


# Training

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

Config

In [7]:
train_master_dir = './train.csv'
img_dir = '/content/train_data'
EMBEDDING_SIZE = 512

Data

In [8]:
train_master = pd.read_csv(train_master_dir)

image_name_list = train_master['id'].values
label_list = train_master['target'].values

In [9]:
x_train, x_val, y_train, y_val = train_test_split(image_name_list, label_list, test_size=0.25, stratify=label_list, random_state=42)

Transformation

In [10]:
from torchvision import transforms


class Transforms():

    def __init__(self):
        
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize(224),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]),
            'val': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]),
            'test': transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]),
            }
    
    def __call__(self, phase, img):
        """
        Parameters
        ----------
        phase : 'train' or 'val' or 'test'
        """
        return self.data_transform[phase](img)

Dataset

In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset


class DefectDataset(Dataset):
    def __init__(self, image_name_list, label_list, img_dir, transform=None, phase=None):
        self.image_name_list = image_name_list
        self.label_list = label_list
        self.img_dir = img_dir
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.image_name_list)

    def __getitem__(self, index):
        image_path = os.path.join(self.img_dir, self.image_name_list[index])
        image = Image.open(image_path)
        image = self.transform(self.phase, image)
        label = self.label_list[index]
        
        return image, label

In [12]:
# train_datasetのテスト
train_dataset = DefectDataset(
    x_train,
    y_train,
    img_dir=img_dir, 
    transform=Transforms(),
    phase='train'
)
print(train_dataset.__getitem__(0)[0].shape)
print('label:', train_dataset.__getitem__(0)[1])
print(train_dataset.__getitem__(0))

torch.Size([3, 224, 224])
label: 0
(tensor([[[0.5364, 0.5536, 0.5536,  ..., 0.8276, 0.8447, 0.8618],
         [0.5364, 0.5536, 0.5536,  ..., 0.8276, 0.8447, 0.8618],
         [0.5364, 0.5536, 0.5536,  ..., 0.8276, 0.8447, 0.8618],
         ...,
         [0.7762, 0.7762, 0.7762,  ..., 1.1187, 1.1015, 1.1015],
         [0.7762, 0.7762, 0.7762,  ..., 1.1187, 1.1015, 1.1015],
         [0.7933, 0.7933, 0.7933,  ..., 1.1187, 1.1015, 1.1015]],

        [[0.6779, 0.6954, 0.6954,  ..., 0.9755, 0.9930, 1.0105],
         [0.6779, 0.6954, 0.6954,  ..., 0.9755, 0.9930, 1.0105],
         [0.6779, 0.6954, 0.6954,  ..., 0.9755, 0.9930, 1.0105],
         ...,
         [0.9230, 0.9230, 0.9230,  ..., 1.2731, 1.2556, 1.2556],
         [0.9230, 0.9230, 0.9230,  ..., 1.2731, 1.2556, 1.2556],
         [0.9405, 0.9405, 0.9405,  ..., 1.2731, 1.2556, 1.2556]],

        [[0.8971, 0.9145, 0.9145,  ..., 1.1934, 1.2108, 1.2282],
         [0.8971, 0.9145, 0.9145,  ..., 1.1934, 1.2108, 1.2282],
         [0.8971, 0.91

Datamodule

In [13]:
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl

from omegaconf import DictConfig


class DefectDataModule(pl.LightningDataModule):
    def __init__(self, x_train, y_train, x_val, y_val, img_dir):
        super().__init__()
        self.x_train = x_train
        self.y_train = y_train
        self.x_val = x_val
        self.y_val = y_val
        self.img_dir = img_dir

    def setup(self, stage=None):
        self.train_dataset = DefectDataset(
            image_name_list=self.x_train,
            label_list=self.y_train,
            img_dir=self.img_dir,
            transform=Transforms(),
            phase='train'
        )
        self.val_dataset = DefectDataset(
            image_name_list=self.x_val,
            label_list=self.y_val,
            img_dir=self.img_dir,
            transform=Transforms(),
            phase='val'
        )

    def train_dataloader(self):   
        return DataLoader(
            self.train_dataset,
            batch_size=4,
            shuffle=True,
            num_workers=2,
            pin_memory=True,
            )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=1,
            shuffle=False,
            num_workers=2,
            pin_memory=True,
            )

In [14]:
datamodule = DefectDataModule(
    x_train=x_train,
    y_train=y_train,
    x_val=x_val,
    y_val=y_val,
    img_dir=img_dir,
)

In [15]:
# datamoduleのテスト
datamodule.setup()

# datamoduleのtrain_datasetをテスト
print(datamodule.train_dataset.__getitem__(0)[0].shape)
print('label:', datamodule.train_dataset.__getitem__(0)[1])
print(datamodule.train_dataset.__getitem__(0))

torch.Size([3, 224, 224])
label: 0
(tensor([[[0.5364, 0.5536, 0.5536,  ..., 0.8276, 0.8447, 0.8618],
         [0.5364, 0.5536, 0.5536,  ..., 0.8276, 0.8447, 0.8618],
         [0.5364, 0.5536, 0.5536,  ..., 0.8276, 0.8447, 0.8618],
         ...,
         [0.7762, 0.7762, 0.7762,  ..., 1.1187, 1.1015, 1.1015],
         [0.7762, 0.7762, 0.7762,  ..., 1.1187, 1.1015, 1.1015],
         [0.7933, 0.7933, 0.7933,  ..., 1.1187, 1.1015, 1.1015]],

        [[0.6779, 0.6954, 0.6954,  ..., 0.9755, 0.9930, 1.0105],
         [0.6779, 0.6954, 0.6954,  ..., 0.9755, 0.9930, 1.0105],
         [0.6779, 0.6954, 0.6954,  ..., 0.9755, 0.9930, 1.0105],
         ...,
         [0.9230, 0.9230, 0.9230,  ..., 1.2731, 1.2556, 1.2556],
         [0.9230, 0.9230, 0.9230,  ..., 1.2731, 1.2556, 1.2556],
         [0.9405, 0.9405, 0.9405,  ..., 1.2731, 1.2556, 1.2556]],

        [[0.8971, 0.9145, 0.9145,  ..., 1.1934, 1.2108, 1.2282],
         [0.8971, 0.9145, 0.9145,  ..., 1.1934, 1.2108, 1.2282],
         [0.8971, 0.91

Model

In [16]:
import torch
import torch.nn as nn
import timm
import pytorch_metric_learning
from pytorch_metric_learning.utils import common_functions


class ConvnextBase(nn.Module):
    def __init__(self):
        super(ConvnextBase, self).__init__()
        self.model_name = 'convnext_base'
        self.pretrained = True

        self.trunk = timm.create_model(
            self.model_name,
            pretrained=self.pretrained,
        )
        self.trunk.head.fc = common_functions.Identity()
        self.embedder = nn.Linear(1024, EMBEDDING_SIZE)
    
    def forward(self, x):
        x = self.trunk(x)
        x = self.embedder(x)
        return x

Losses

In [17]:
from pytorch_metric_learning import losses, distances, regularizers


def get_loss_fn(loss_name):
    if loss_name == 'ArcFaceLoss':
        distance = distances.CosineSimilarity()
        regularizer = regularizers.RegularFaceRegularizer()
        loss_fn = losses.ArcFaceLoss(
            num_classes=2,
            embedding_size=EMBEDDING_SIZE,
            margin=28.6,
            scale=64,
            weight_regularizer=regularizer, 
            distance=distance
        )
        return loss_fn
    
    elif loss_name == 'SubCenterArcFaceLoss':
        loss_fn = losses.SubCenterArcFaceLoss(
            num_classes=2, 
            embedding_size=EMBEDDING_SIZE, 
            margin=28.6, 
            scale=64, 
            sub_centers=3,
        )
        return loss_fn
    
    else:
        raise ValueError(f'Unknown optimizer: {loss_name}')

Optimizer

In [18]:
import torch
from torch import optim


def get_optimizer(net: nn.Module):
    return optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-5)

Net

In [19]:
import torch
import torch.nn.functional as F

import pytorch_lightning as pl
import torchmetrics

import timm


class Net(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # model
        self.model = ConvnextBase()
        
        # loss function
        self.loss_fn = get_loss_fn(loss_name='ArcFaceLoss')

        # Important: This property activates manual optimization.
        self.automatic_optimization = False
    

    def forward(self, x):
        return self.model(x)
    

    def training_step(self, batch, batch_idx):
        optimizer, loss_optimizer = self.optimizers()
        optimizer.zero_grad()
        loss_optimizer.zero_grad()

        x, t = batch
        y = self(x)
        loss = self.loss_fn(y, t)

        # instead of loss.backward()
        self.manual_backward(loss)

        optimizer.step()
        loss_optimizer.step()

        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        return {'loss': loss}
    

    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = self.loss_fn(y, t)

        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

        return {'loss': loss}
    

    def configure_optimizers(self):
        optimizer = get_optimizer(net=self.model)
        loss_optimizer = get_optimizer(net=self.loss_fn)
        return [optimizer, loss_optimizer]

In [20]:
pl.seed_everything(0)

INFO:lightning_lite.utilities.seed:Global seed set to 0


0

In [21]:
net = Net()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth


In [22]:
trainer = pl.Trainer(max_epochs=50)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [ ]:
trainer.fit(net, datamodule=datamodule)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type         | Params
-----------------------------------------
0 | model   | ConvnextBase | 88.1 M
1 | loss_fn | ArcFaceLoss  | 1.0 K 
-----------------------------------------
88.1 M    Trainable params
0         Non-trainable params
88.1 M    Total params
352.369   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]